## Group Project - Investment Case Study
   #### Group Members:
    -  Vinayak Bandhu
    -  Sarathbabu Sankaran
    -  Puneet Agarwal
    -  Rakesh Gorai

### Checkpoint 1 : Data Cleaning

In [ ]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import sys,Numpy,pandas,chardet packages

import numpy as np
import pandas as pd

In [ ]:
# Reading data from different data files

companies = pd.read_csv('companies.txt', sep='\t', encoding='ISO-8859-1')
rounds2 = pd.read_csv('rounds2.csv', encoding='ISO-8859-1')
mapping = pd.read_csv('mapping.csv')

In [ ]:
#Cleaning Data in Dataframes (removing special characters)

companies['permalink'] = companies.permalink.str.encode('utf-8').str.decode('ascii','ignore').str.lower()
rounds2['company_permalink'] = rounds2.company_permalink.str.encode('utf-8').str.decode('ascii','ignore').str.lower()
companies['name'] = companies.name.str.encode('utf-8').str.decode('ascii', 'ignore').str.lower()

In [ ]:
# Unique companies present in rounds2 and companies files:

print(rounds2['company_permalink'].nunique())
print(companies['permalink'].nunique())

In [ ]:
# Inspect companies dataframe 

print(companies.shape)
print(companies.info())
print(companies.describe())

In [ ]:
# Inspect rounds2 dataframe

print(rounds2.shape)
print(rounds2.info())
print(rounds2.describe(include = 'all'))

In [ ]:
# Check to see which column can be used as unique key in companies dataframe

## Check for non-null
print(companies.isnull().sum())

## Check for Number of rows and columns:
print(companies.shape)

## Check for Number of unique values in each columns:
print(companies.nunique())

In [ ]:
# Check whether any company in rounds2 which is not present in companies dataframe

companies.loc[~companies['permalink'].isin(rounds2['company_permalink']),:]

In [ ]:
# Merge the companies and rounds2 dataframe. Since the name of common column in both the dataframe is not same,
# the name of company_permalink is renamed to permalink to remove the duplication of column in master_frame

master_frame = pd.merge(rounds2.rename(columns={'company_permalink':'permalink'}), companies, how = 'inner', on = 'permalink')

In [ ]:
# No of observations in master_frame

master_frame.shape[0]

In [ ]:
# Inspect the master_dataframe

print(master_frame.shape)
print(master_frame.describe)
print(master_frame.info())

#### Cleaning the master_frame dataset

In [ ]:
# Storing the master_frame Dataset into a temp dataframe to check to retained rows percentage after cleaning the data
master_frame_temp=master_frame

# calculating number of null values in columns and their percentage
print(master_frame.isnull().sum())

round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

In [ ]:
# Drop unnecessary columns with high percentage of null values considering above 10%

master_frame.drop(['funding_round_code','founded_at'],axis=1,inplace=True)

round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

In [ ]:
# Remove rows with total number of null values 7 or more
master_frame=master_frame[(master_frame.isnull().sum(axis=1))<7]

# Remove rows with raised_amount_usd as null
master_frame=master_frame[~master_frame.raised_amount_usd.isnull()]

# Remove rows with null value country codes
master_frame=master_frame[~master_frame.country_code.isnull()]

round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2)

In [ ]:
# Check number of retained rows

round((master_frame.shape[0]/master_frame_temp.shape[0])*100,2)

In [ ]:
# Converting the raised_amount_usd from usd to usd million

master_frame[['raised_amount_usd']]=round(master_frame[['raised_amount_usd']].apply(lambda x:x/1000000),2)

In [ ]:
# Average funding amount in Venture,Private equity, angel and seed type investments

group_by_funding_round_type = master_frame.groupby('funding_round_type')
average_fund = pd.DataFrame(group_by_funding_round_type['raised_amount_usd'].mean())

#round(master_frame.loc[master_frame['funding_round_type']=='angel'].mean(),2)
round(average_fund.loc[['venture','angel','seed','private_equity']],3).sort_values(by = 'raised_amount_usd', ascending = False)

Here the `venture` is the second highest investment type with raised_amount_mn_usd and falls under the 5 to 15 M USD window of Spark Funds. Hence this is the most suitable investment type for Spark Funds.

### Checkpoint 3: Country Analysis

In [ ]:
# Top nine countries(based on the total investment amount each country has received)

#Create temporary dataframe to contain only venture type investments data
venture_investment_frame = master_frame.loc[master_frame['funding_round_type'] == 'venture']

#Create top9 dataframe by grouping the venture_investment_dataframe
group_by_country=venture_investment_frame.groupby('country_code')
top9 = group_by_country['raised_amount_usd'].sum().sort_values(ascending=False).head(9)
print(top9)

Here the `USA`, `GBR` and `IND` are the Top 3 English speaking countries in venture venture type investment. Hence are the most suitable countries where Sparks funds can invest.

### Checkpoint 4: Sector Analysis 1

In [ ]:
# Create sector_frame to contain data from countries 'USA', 'GBR' and 'IND'

sector_frame=venture_investment_frame.loc[(venture_investment_frame['country_code'] == 'USA') | (venture_investment_frame['country_code'] == 'GBR') | (venture_investment_frame['country_code'] == 'IND')]

# Create a column primamry_sector by spliting category_list on '|' and taking the left value
sector_frame['primary_sector'] = (sector_frame['category_list'].str.split('|',  expand = True)).iloc[:,0]
sector_frame

In [ ]:
# Inspect mapping file for null values

len(mapping.index)
print(mapping.isnull().sum())
print(round(100*(mapping.isnull().sum()/len(mapping.index)), 2))

#### Cleaning the mapping frame

In [ ]:
# Cleaning the mapping dataframe. The 'na' in each category is replaced with '0' in the dataframe.
# hence changing the '0' to 'na' again.

mapping['category_list'] = mapping['category_list'].str.replace('0', 'na', regex=False)
mapping.head(10)

In [ ]:
# Normalizing the mapping dataframe so that merge operation can be done appropriately.

mapping = pd.melt(mapping, id_vars = ["category_list"], var_name = "main_sector")
mapping.head(10)

In [ ]:
# Removing the unwanted rows which doesn't provide any mapping information
mapping = mapping[~(mapping['value'] == 0)]

# Dropping the value column, since it is not useful.
mapping.drop(['value'],axis=1,inplace=True)
mapping.head(10)

In [ ]:
# Merging the sector data with main sector mapping

sector_frame = pd.merge(sector_frame,mapping.rename(columns={'category_list':'primary_sector'}), how = 'inner', on = 'primary_sector' )

# Dropping rows with primary_sector value null
sector_frame= sector_frame[~sector_frame.primary_sector.isnull()]
sector_frame.head(10)

### Checkpoint 5: Sector Analysis 2

In [ ]:
# Data frame D1 for country USA and funding amount between 5 and 15 M USD.

# Two new columns are added total_amount_invested and total_number_of_investments
D1 = sector_frame.loc[(sector_frame['country_code'] == 'USA') & (sector_frame['raised_amount_usd'] >= 5) & (sector_frame['raised_amount_usd'] <= 15)]
D1_groupBy = D1.groupby('main_sector')
D1['total_amount_invested'] = D1_groupBy['raised_amount_usd'].transform('sum')
D1['total_number_of_investments'] = D1_groupBy['raised_amount_usd'].transform('count')
D1

In [ ]:
# Data frame D2 for country GBR and funding amount between 5 and 15 M USD.

# Two new columns are added total_amount_invested and total_number_of_investments
D2 = sector_frame.loc[(sector_frame['country_code'] == 'GBR') & (sector_frame['raised_amount_usd'] >= 5) & (sector_frame['raised_amount_usd'] <= 15)]
D2_groupBy = D2.groupby('main_sector')
D2['total_amount_invested'] = D2_groupBy['raised_amount_usd'].transform('sum')
D2['total_number_of_investments'] = D2_groupBy['raised_amount_usd'].transform('count')
D2

In [ ]:
# Data frame D3 for country IND and funding amount between 5 and 15 M USD.

# Two new columns are added total_amount_invested and total_number_of_investments
D3 = sector_frame.loc[(sector_frame['country_code'] == 'IND') & (sector_frame['raised_amount_usd'] >= 5) & (sector_frame['raised_amount_usd'] <= 15)]
D3_groupBy = D3.groupby('main_sector')
D3['total_amount_invested'] = D3.groupby('main_sector')['raised_amount_usd'].transform('sum')
D3['total_number_of_investments'] = D3.groupby('main_sector')['raised_amount_usd'].transform('count')
D3

In [ ]:
#Total number of investments (count) and Total amount of investment (USD) in USA

print(D1['raised_amount_usd'].count())
print(round(D1['raised_amount_usd'].sum(),2))

In [ ]:
#Total number of investments (count) and Total amount of investment (USD) in GBR

print(D2['raised_amount_usd'].count())
print(round(D2['raised_amount_usd'].sum(),2))

In [ ]:
#Total number of investments (count) and Total amount of investment (USD) in IND

print(D3['raised_amount_usd'].count())
print(round(D3['raised_amount_usd'].sum(),2))

In [ ]:
# Counting number of investments in USA based on raised_amount_usd 

D1_groupBy['raised_amount_usd'].count().sort_values(ascending = False).head(3)

The top 3 main_sector based on the count of investments in USA are : `Others`, `Social, Finance, Analytics, Advertising` and `Cleantech / Semiconductors`

In [ ]:
# Counting number of investments in GBR based on raised_amount_usd 

D2_groupBy['raised_amount_usd'].count().sort_values(ascending = False).head(3)

The top 3 main_sector based on the count of investments in GBR are : `Others`, `Social, Finance, Analytics, Advertising` and `Cleantech / Semiconductors`

In [ ]:
# Counting number of investments in IND based on raised_amount_usd

D3_groupBy['raised_amount_usd'].count().sort_values(ascending = False).head(3)

The top 3 main_sector based on the count of investments in IND are : `Others`, `Social, Finance, Analytics, Advertising` and `News, Search and Messaging`

In [ ]:
#Name of the company in USA for "Others" Sector, which is top in amount_invested

D1_groupby_name=D1[D1.main_sector=='Others'].groupby('name')
D1_groupby_name['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

In [ ]:
#Name of the company in GBR for "Others" Sector, which is top in amount invested

D2_groupby_name=D2[D2.main_sector=='Others'].groupby('name')
D2_groupby_name['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

In [ ]:
#Name of the company in IND for "Others" Sector, which is top in amount invested

D3_groupby_name=D3[D3.main_sector=='Others'].groupby('name')
D3_groupby_name['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

In [ ]:
#Name of the company in USA for "Social, Finance, Analytics, Advertising" Sector, which is top in amount invested.

D1_groupby_name1=D1[D1.main_sector=='Social, Finance, Analytics, Advertising'].groupby('name')
D1_groupby_name1['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

In [ ]:
#Name of the company in GBR for "Social, Finance, Analytics, Advertising" Sector, which is top in amount invested.

D2_groupby_name2=D2[D2.main_sector=='Social, Finance, Analytics, Advertising'].groupby('name')
D2_groupby_name2['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

In [ ]:
#Name of the company in IND for "Social, Finance, Analytics, Advertising" Sector, which is top in amount invested.

D3_groupby_name3=D3[D3.main_sector=='Social, Finance, Analytics, Advertising'].groupby('name')
D3_groupby_name3['raised_amount_usd'].sum().sort_values(ascending = False).head(5)

### Checkpoint 6: Plots

In [ ]:
# Exporting data to Excel to create a tableau connection to plot graphs

master_frame.to_excel(pd.ExcelWriter('Master_Frame_Data.xlsx'),'master_frame_data')
sector_frame.to_excel(pd.ExcelWriter('Sector_Frame_Data.xlsx'),'sector_frame_data')

In [ ]:
# Analyse excel for the exported dataframes and, 
# match it with results below to verify proper export of master_frame and sector_frame

# master_frame validation
print(master_frame.shape)
print(master_frame.info())

# sector_frame_validation
print(sector_frame.shape)
print(sector_frame.info())